In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np

In [21]:
data = pd.read_csv('/Users/annafurgala-wojas/New_project/Heart-disease-dataset/healthcare-dataset-stroke-data.csv')
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [22]:
data.groupby("stroke")["bmi"].apply(lambda x: x.isna().sum())

stroke
0    161
1     40
Name: bmi, dtype: int64

In [23]:
data["bmi"] = data.groupby("stroke")["bmi"].transform(
    lambda x: x.fillna(x.median())
)

In [24]:
data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.871409,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.699951,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.800000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.000000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,32.800000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [25]:
data = data.drop(columns=["id"])

In [26]:

data_final = pd.get_dummies(data, columns=["gender", "ever_married", "work_type", "Residence_type", "smoking_status"], dtype=int)

data_final.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
1,61.0,0,0,202.21,29.7,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0


In [27]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X = data_final.drop(columns=["stroke"])
y = data_final['stroke']

In [28]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE

# --- Dane: bardzo niezbalansowane (1% pozytywna klasa) ---
X, y = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    weights=[0.99, 0.01],
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- SMOTE: oversampling klasy rzadkiej ---
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# --- scale_pos_weight ---
scale_pos_weight = sum(y_train_res == 0) / sum(y_train_res == 1)

def find_threshold_for_recall(y_true, y_prob, min_recall=0.7):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    # znajdź pierwszy threshold, gdzie recall >= min_recall
    idxs = np.where(recall >= min_recall)[0]
    if len(idxs) == 0:
        return None, precision, recall, None  # brak progu spełniającego warunek
    best_idx = idxs[-1]  # weź próg z najwyższą precyzją przy tym recall
    return thresholds[best_idx], precision, recall, best_idx

# --- 3. CatBoost ---
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    eval_metric='Recall',
    class_weights=[1, scale_pos_weight],
    random_state=42,
    verbose=False
)

cat_model.fit(X_train_res, y_train_res)
y_prob_cat = cat_model.predict_proba(X_test)[:, 1]

thr_rec_cat, _, _, idx_rec_cat = find_threshold_for_recall(y_test, y_prob_cat, min_recall=0.7)
y_pred_rec_cat = (y_prob_cat > thr_rec_cat).astype(int) if thr_rec_cat else None


print("\n=== CatBoost ===")
if y_pred_rec_cat is not None:
    print("Threshold dla recall>=0.7:", round(thr_rec_cat, 4))
    print(classification_report(y_test, y_pred_rec_cat, digits=4))


/Users/annafurgala-wojas/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(



=== CatBoost ===
Threshold dla recall>=0.7: 0.0608
              precision    recall  f1-score   support

           0     0.9947    0.8550    0.9196      1972
           1     0.0623    0.6786    0.1141        28

    accuracy                         0.8525      2000
   macro avg     0.5285    0.7668    0.5168      2000
weighted avg     0.9816    0.8525    0.9083      2000



In [30]:
import joblib

feature_order = list(range(X_test.shape[1]))  # numery kolumn
joblib.dump((cat_model, feature_order), "Selected_model.pkl")


['Selected_model.pkl']